## Parts of this Notebook was shamelessly copied from Scott Henderson (https://github.com/geohackweek/tutorial_contents/blob/master/raster/notebooks/rasterio-landsat-aws.ipynb)

In [ ]:
#First we import some packages

import os
import rasterio
import rasterio.plot
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import boto3

In [ ]:
#Then we import images from the Landsat on AWS public s3 - similar to what we did when we copied data from bucket to bucket

red_image = "s3://landsat-pds/c1/L8/042/034/LC08_L1TP_042034_20170616_20170629_01_T1/LC08_L1TP_042034_20170616_20170629_01_T1_B4.TIF"
nir_image = "s3://landsat-pds/c1/L8/042/034/LC08_L1TP_042034_20170616_20170629_01_T1/LC08_L1TP_042034_20170616_20170629_01_T1_B5.TIF"

In [ ]:
# We will quickly look at the red band image with rasterio

with rasterio.open(red_image) as src:
    profile = src.profile
    oviews = src.overviews(1) # list of overviews from biggest to smallest
    oview = oviews[1]  # Use second-highest resolution overview
    print('Decimation factor= {}'.format(oview))
    red = src.read(1, out_shape=(1, int(src.height // oview), int(src.width // oview)))
    
plt.imshow(red)
plt.colorbar()
plt.title('{}\nRed {}'.format(red_image, red.shape))
plt.xlabel('Column #')
plt.ylabel('Row #')

In [ ]:
#Let's also look at the near infrared band

with rasterio.open(nir_image) as src:
    oviews = src.overviews(1) # list of overviews from biggest to smallest
    oview = oviews[1]  # Use second-highest resolution overview
    nir = src.read(1, out_shape=(1, int(src.height // oview), int(src.width // oview)))
    
plt.imshow(nir)
plt.colorbar()
plt.title('{}\nNIR {}'.format(nir_image, nir.shape))
plt.xlabel('Column #')
plt.ylabel('Row #')

In [ ]:
#Quickly calculating the NDVI 
def calc_ndvi(nir,red):
    '''Calculate NDVI from integer arrays'''
    nir = nir.astype('f4')
    red = red.astype('f4')
    ndvi = (nir - red) / (nir + red)
    return ndvi

ndvi = calc_ndvi(nir,red)
plt.imshow(ndvi, cmap='RdYlGn')
plt.colorbar()
plt.title('NDVI - 2017-06-16')
plt.xlabel('Column #')
plt.ylabel('Row #')


In [ ]:
# Let's try to write this back to an s3 bucket. First we save this file 

localname = 'LC08_L1TP_042034_20170616_20170629_01_T1_NDVI_OVIEW.tif'

with rasterio.open(nir_image) as src:
    profile = src.profile.copy()
    
    aff = src.transform
    newaff = rasterio.Affine(aff.a * oview, aff.b, aff.c,
                             aff.d, aff.e * oview, aff.f)
    profile.update({
            'dtype': 'float32',
            'height': ndvi.shape[0],
            'width': ndvi.shape[1],
            'transform': newaff})  
    
    with rasterio.open(localname, 'w', **profile) as dst:
        dst.write_band(1, ndvi)

In [ ]:
#Using boto3 with is the Amazon Web Services (AWS) Software Development Kit (SDK) for Python, we will upload the file and list the bucket contents
s3 = boto3.resource('s3')

s3.meta.client.upload_file('LC08_L1TP_042034_20170616_20170629_01_T1_NDVI_OVIEW.tif', 'esip-esipuser', 'LC08_L1TP_042034_20170616_20170629_01_T1_NDVI_OVIEW.tif')

for obj in s3.Bucket(name='esip-esipuser').objects.all():
    print(os.path.join(obj.bucket_name, obj.key))